In [64]:
import pandas as pd
import pygad as pg
import numpy as np
import random
import time


In [65]:
def get_inputs(inputs_file):
    result = {}
    with open(inputs_file) as inputs:
        nr = 1
        for line in inputs:
            line = line[:-1].split(";")
            result[nr] = [int(number) for number in line]
            nr += 1
    return result


In [66]:
small_inputs = get_inputs("small_inputs.txt")
medium_inputs = get_inputs("medium_inputs.txt")
big_inputs = get_inputs("big_inputs.txt")


In [67]:
def get_result(solution):
    res = []
    for i in range(len(solution)):
        if solution[i] == 1:
            res.append(set[i])
    return res


def fitness_func(solution, solution_idx):
    if np.count_nonzero(np.array(solution) == 1) == 0:
        return -np.Inf
    else:
        res = np.sum(np.multiply(set, solution))
        return -abs(res)


set = [-7, -3, -2, 9000, 5, 8]
solution = [0, 1, 1, 0, 1, 0]

fitness = fitness_func(solution, 0)

result = get_result(solution)

print("Zbiór początkowy: {}".format(set))
print("Znaleziony podzbiór: {}".format(result))
print("Suma wszystkich elementów podzbioru: {}".format(fitness))


Zbiór początkowy: [-7, -3, -2, 9000, 5, 8]
Znaleziony podzbiór: [-3, -2, 5]
Suma wszystkich elementów podzbioru: 0


In [68]:
gene_space = [0, 1]
num_genes = len(set)

parent_selection_type = "sss"
crossover_type = "single_point"
mutation_type = "random"
mutation_num_genes = 1
stop_criteria = "reach_0"


In [69]:
def create_ga_instance(num_genes, sol_per_pop, num_parents_mating, num_generations, keep_parents):
    return pg.GA(
        gene_space=gene_space,
        num_generations=num_generations,
        num_parents_mating=num_parents_mating,
        fitness_func=fitness_func,
        sol_per_pop=sol_per_pop,
        num_genes=num_genes,
        parent_selection_type=parent_selection_type,
        keep_parents=keep_parents,
        crossover_type=crossover_type,
        mutation_type=mutation_type,
        mutation_num_genes=mutation_num_genes,
        stop_criteria=stop_criteria
    )


In [70]:
def show_result(time, fitness):
    print("Czas wykonania algorytmu: {} sekund".format(time))
    print("Suma elementów znalezionego podzbioru: {} => {}"
          .format(int(fitness), "rozwiązano problem" if fitness == 0 else "nie rozwiązano problemu"))


In [71]:
set = small_inputs[random.randint(1, 5)]

start = time.time()

small_instance = create_ga_instance(
    num_genes=100,
    sol_per_pop=20,
    num_parents_mating=10,
    num_generations=100,
    keep_parents=6
)

small_instance.run()

end = time.time()

best_solution, fitness, _ = small_instance.best_solution()

show_result(end - start, fitness)


Czas wykonania algorytmu: 0.02973175048828125 sekund
Suma elementów znalezionego podzbioru: 0 => rozwiązano problem


In [72]:
set = medium_inputs[random.randint(1, 5)]

start = time.time()

medium_instance = create_ga_instance(
    num_genes=1000,
    sol_per_pop=100,
    num_parents_mating=50,
    num_generations=300,
    keep_parents=30
)

medium_instance.run()

end = time.time()

best_solution, fitness, _ = medium_instance.best_solution()

show_result(end - start, fitness)


Czas wykonania algorytmu: 0.3085916042327881 sekund
Suma elementów znalezionego podzbioru: 0 => rozwiązano problem


In [73]:
set = big_inputs[random.randint(1, 5)]

start = time.time()

big_instance = create_ga_instance(
    num_genes=5000,
    sol_per_pop=200,
    num_parents_mating=100,
    num_generations=500,
    keep_parents=50
)

big_instance.run()

end = time.time()

best_solution, fitness, _ = big_instance.best_solution()

show_result(end - start, fitness)


Czas wykonania algorytmu: 22.364306688308716 sekund
Suma elementów znalezionego podzbioru: 0 => rozwiązano problem


In [74]:
ga_times = pd.read_csv("ga_times.csv")


def ga_data(input):
    column = ga_times[ga_times[input] != 0][input]
    result = {}
    result["avg_time"] = column.mean()
    result["accuraccy"] = len(column) / 100
    result["min_time"] = column.min()
    result["max_time"] = column.max()
    return result


In [75]:
small_inputs_results = ga_data("small_input")
print(small_inputs_results)


{'avg_time': 0.021550991535186723, 'accuraccy': 1.0, 'min_time': 0.00447678565979, 'max_time': 0.0682284832000732}


In [76]:
medium_inputs_results = ga_data("medium_input")
print(medium_inputs_results)


{'avg_time': 0.8110143566131591, 'accuraccy': 1.0, 'min_time': 0.0252683162689209, 'max_time': 3.8786752223968506}


In [77]:
big_inputs_results = ga_data("big_input")
print(big_inputs_results)


{'avg_time': 16.343913774297693, 'accuraccy': 0.99, 'min_time': 0.1766080856323242, 'max_time': 48.67165207862854}


| Rozmiar danych wejściowych |         Małe          |       Średnie       |        Duże         |
| :------------------------: | :-------------------: | :-----------------: | :-----------------: |
|   Skuteczność algorytmu    |         100%          |        100%         |         99%         |
|    Najszybsze wykonanie    |   0.00447678565979s   | 0.0252683162689209s | 0.1766080856323242s |
|    Najdłuższe wykonanie    |  0.0682284832000732s  | 3.8786752223968506s | 48.67165207862854s  |
|   Średni czas wykonania    | 0.021550991535186723s | 0.8110143566131591s | 16.343913774297693s |


In [78]:
from pyswarms.discrete.binary import BinaryPSO


In [79]:
def optimizer_function(x):
    n_particles = x.shape[0]
    j = [-fitness_func(x[i], _) for i in range(n_particles)]
    return np.array(j)


In [80]:
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 10, 'p': 1}


In [81]:
set = small_inputs[random.randint(1, 5)]

start = time.time()

optimizer = BinaryPSO(n_particles=10, dimensions=len(set), options=options)

cost, pos = optimizer.optimize(optimizer_function, iters=500, verbose=False)

end = time.time()

show_result(end - start, cost)


Czas wykonania algorytmu: 0.1991121768951416 sekund
Suma elementów znalezionego podzbioru: 0 => rozwiązano problem


In [82]:
set = medium_inputs[random.randint(1, 5)]

options['k'] = 30

start = time.time()

optimizer = BinaryPSO(n_particles=30, dimensions=len(set), options=options)

cost, pos = optimizer.optimize(optimizer_function, iters=1250, verbose=False)

end = time.time()

show_result(end - start, cost)


Czas wykonania algorytmu: 6.19307279586792 sekund
Suma elementów znalezionego podzbioru: 0 => rozwiązano problem


In [83]:
set = big_inputs[random.randint(1, 5)]

options['k'] = 50

start = time.time()

optimizer = BinaryPSO(n_particles=50, dimensions=len(set), options=options)

cost, pos = optimizer.optimize(optimizer_function, iters=1250, verbose=False)

end = time.time()

show_result(end - start, cost)


Czas wykonania algorytmu: 66.82745933532715 sekund
Suma elementów znalezionego podzbioru: 8 => nie rozwiązano problemu


In [85]:
pso_times = pd.read_csv("pso_times.csv")


def pso_data(input):
    column = pso_times[pso_times[input] != 0][input]
    result = {}
    result["avg_time"] = column.mean()
    result["accuraccy"] = len(column) / 100
    result["min_time"] = column.min()
    result["max_time"] = column.max()
    return result


In [86]:
print(pso_data("small_input"))
print(pso_data("medium_input"))
print(pso_data("big_input"))


{'avg_time': 0.22535311222076415, 'accuraccy': 1.0, 'min_time': 0.1616637706756591, 'max_time': 0.3144810199737549}
{'avg_time': 5.390086862776014, 'accuraccy': 0.72, 'min_time': 4.791399717330933, 'max_time': 6.987470388412476}
{'avg_time': 47.398172673057104, 'accuraccy': 0.17, 'min_time': 45.748215675354, 'max_time': 49.09682369232178}


| Rozmiar danych wejściowych |         Małe         |      Średnie       |        Duże         |
| :------------------------: | :------------------: | :----------------: | :-----------------: |
|   Skuteczność algorytmu    |         100%         |        72%         |         17%         |
|    Najszybsze wykonanie    | 0.1616637706756591s  | 4.791399717330933s |  45.748215675354s   |
|    Najdłuższe wykonanie    | 0.3144810199737549s  | 6.987470388412476s | 49.09682369232178s  |
|   Średni czas wykonania    | 0.22535311222076415s | 5.390086862776014s | 47.398172673057104s |
